# Practica 25/26

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [92]:
df = pd.read_csv('./salario.csv')

In [93]:
df.sample(5)

,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
5809,39,Private,Some-college,Divorced,Sales,Soltero-a,Blanco,Mujer,0,0,40,United-States,<=50K
4388,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Marido,Blanco,Hombre,0,0,55,United-States,<=50K
3485,55,Private,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,40,United-States,>50K
10115,21,Private,HS-grad,Married-civ-spouse,Adm-clerical,Marido,Blanco,Hombre,0,0,40,United-States,<=50K
793,23,Private,HS-grad,Never-married,Sales,No-en-familia,Blanco,Mujer,2463,0,40,United-States,<=50K


Posibles valores nulos

In [94]:
for column in df.columns:
    print(df[column].value_counts())

edad
31    764
23    763
36    762
35    756
34    754
     ... 
84     10
83      6
88      3
85      2
86      1
Name: count, Length: 72, dtype: int64
trabajo
Private             19483
Self-emp-not-inc     2211
Local-gov            1810
?                    1568
State-gov            1119
Self-emp-inc          961
Federal-gov           830
Without-pay            11
Never-worked            5
Name: count, dtype: int64
estudios
HS-grad         9033
Some-college    6250
Bachelors       4642
Masters         1477
Assoc-voc       1184
11th            1023
Assoc-acdm       905
10th             811
7th-8th          563
Prof-school      488
9th              438
12th             369
Doctorate        357
5th-6th          282
1st-4th          131
Preschool         45
Name: count, dtype: int64
estado-civil
Married-civ-spouse       12834
Never-married             9172
Divorced                  3837
Separated                  899
Widowed                    869
Married-spouse-absent      366
Married-A

Reemplazamos '?' por np.nan

In [95]:
df_null = df.replace(r'.*\?.*', np.nan, regex=True)

Porcentaje de nulos por columna

In [96]:
(df_null.isnull().sum(axis=0) / df_null.shape[0] * 100).sort_values(ascending=False)

trabajo.1                5.618258
trabajo                  5.600400
pais-origen              1.767983
estudios                 0.000000
edad                     0.000000
estado-civil             0.000000
posicion-familiar        0.000000
sexo                     0.000000
etnia                    0.000000
ganancias_inversiones    0.000000
perdidas_inversiones     0.000000
horas-trabajo_semana     0.000000
salario                  0.000000
dtype: float64

Porcentaje de nulos por fila

In [97]:
(df_null.isnull().sum(axis=1) / df_null.shape[1] * 100).sort_values(ascending=False)

61       23.076923
7862     23.076923
6059     23.076923
3579     23.076923
16838    23.076923
           ...    
57        0.000000
58        0.000000
59        0.000000
60        0.000000
7         0.000000
Length: 27998, dtype: float64